In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType

In [6]:
from pyspark.sql import functions as ff

In [7]:
from pyspark.ml import Pipeline, PipelineModel

In [8]:
globals()['models_loaded'] = False

#def predict(df):
    # Replace this with something smarter
#    if any([x in df.review_text for x in ['not', 'bad', 'terrible', 'zero', 'one']]):
#        return 1
#    return 5

#predict_udf = udf(predict, IntegerType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    df_withpreds = df.withColumn('text', ff.concat(ff.col('review_title'), ff.lit(' '), ff.col('review_text'))) #removed book_title
    
    drop_list = ['book_title', 'review_title', 'review_user', 'book_id', 'review_id', 
             'timestamp', 'review_text']
    df_withpreds = df_withpreds.select([column for column in df_withpreds.columns if column not in drop_list])
    #df_withpreds.show()
    
    # Utilize our predict function
    #df_withpreds = df.withColumn("pred", predict_udf(struct([df[x] for x in df.columns])))
    #df_withpreds.show()
    
    # Normally, you wouldn't use a Python function to predict
    # But an MLlib model you've built and saved with Spark
    
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = PipelineModel.load('pipeline_nb') # Replace this with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # Predict using the model: 
    df_result = globals()['my_model'].transform(df_withpreds)
    #drop_list2 = ['words', 'filtered', 'rawFeatures', 'features', 'label', 
    #         'rawPrediction', 'prediction']
    #df_result = df_result.select([column for column in df_result.columns if column not in drop_list2])
    
    df_result = df_result.select("text","probability","review_score","predictedScore")
    df_result.show()
    #df_result = globals()['my_model'].transform(df)
    #df_result.show()

In [9]:
ssc = StreamingContext(sc, 10)

In [10]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [11]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2019-06-01 17:06:00 =========
+----------+--------------------+--------------+------------+--------------------+--------------------+-----------+----------+
|   book_id|          book_title|     review_id|review_score|         review_text|        review_title|review_user| timestamp|
+----------+--------------------+--------------+------------+--------------------+--------------------+-----------+----------+
|1476773092|Unfreedom of the ...|R3I34CXTG21TUO|           5|While I felt the ...|Compelling, well-...|Serenity...|1559401129|
+----------+--------------------+--------------+------------+--------------------+--------------------+-----------+----------+

+--------------------+--------------------+------------+--------------+
|                text|         probability|review_score|predictedScore|
+--------------------+--------------------+------------+--------------+
|Compelling, well-...|[0.01015436416008...|           5|             4|
+--------------------+--------------